In [ ]:

# Initialize the model
# Use Dask's `map_blocks` to apply the gradient computation function to each chunk of data
def gradient_task(a, b):
    device="cuda:0"
    print(torch.cuda.current_device())
    model_config = AutoConfig.from_pretrained(checkpoint_path)
    model = RobertaForMaskedLM(config=model_config).to(device)
    model.train()
    print(len(a), len(b))
    gradients_a = torch.stack([get_loss_gradient(model, example,device).detach().cpu().flatten() for example in a])
    gradients_b = torch.stack([get_loss_gradient(model, example,device).detach().cpu().flatten() for example in b])
    print(gradients_a.shape, gradients_b.shape)
    print("torch.matmul(gradients_a, gradients_b.T).numpy()", torch.matmul(gradients_a, gradients_b.T).numpy().shape)
    return torch.matmul(gradients_a, gradients_b.T).numpy()
# Apply the gradient computation function across the Dask arrays

gradients = da.map_blocks(gradient_task, input_data, target_data, dtype=np.float32)

# Trigger computation to get the gradients
computed_gradients = gradients.compute()

# Print out the gradients
print(computed_gradients.shape)

# Optionally, you can shut down the Dask client after the computation is complete
# client.close()
torch.save(computed_gradients, "computed_gradients")
client.close()